In [19]:
import pandas as pd
import pyarrow
from kafka import KafkaProducer
from datetime import datetime
import json
import time
from time import sleep


In [20]:
# Custom serializer for handling non-serializable data types
def json_serializer(data):
    def convert(o):
        if isinstance(o, datetime):
            return o.__str__()
    return json.dumps(data, default=convert).encode('utf-8')

In [21]:
# Initialize Kafka Producer
producer = KafkaProducer(bootstrap_servers='localhost:9092', 
                         value_serializer= json_serializer,
                         )

In [22]:
# Path to your Parquet file
parquet_file_path = './data/yellow_tripdata_2024-01.parquet'
df = pd.read_parquet(parquet_file_path)
df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [23]:
# Sample one record and convert it to a dictionary
record = df.sample(n=1).to_dict(orient='records')[0]
record

{'VendorID': 1,
 'tpep_pickup_datetime': Timestamp('2024-01-19 15:37:51'),
 'tpep_dropoff_datetime': Timestamp('2024-01-19 15:47:24'),
 'passenger_count': 2.0,
 'trip_distance': 0.8,
 'RatecodeID': 1.0,
 'store_and_fwd_flag': 'N',
 'PULocationID': 161,
 'DOLocationID': 230,
 'payment_type': 1,
 'fare_amount': 9.3,
 'extra': 2.5,
 'mta_tax': 0.5,
 'tip_amount': 2.65,
 'tolls_amount': 0.0,
 'improvement_surcharge': 1.0,
 'total_amount': 15.95,
 'congestion_surcharge': 2.5,
 'Airport_fee': 0.0}

In [24]:
while True:
    data = df.sample(n=1).to_dict(orient='records')[0]
    producer.send('trips_data', value=data)
    sleep(2)

KeyboardInterrupt: 

In [ ]:
# Clean Up
producer.flush()